In [2]:
import numpy as np
import tensorflow as tf
import os
from tqdm import tqdm
import sqlite3

# tf.enable_eager_execution()

print('All libraries imported successfully')

ModuleNotFoundError: No module named 'numpy'

In [ ]:
def get_images_list(images_path = './data/images/'):
    """Return list of paths to files within provided path"""
    image_list = sorted([images_path + image for image in os.listdir(images_path)])
    return image_list

In [ ]:
def decode_according_to_extension(img_raw,extension='.jpg'):
    if (extension=='.jpg') or (extension=='.jpeg'):
        return tf.image.decode_jpeg(img_raw, channels=3)
    elif (extension=='.png'):
        return tf.image.decode_png(img_raw)
    elif (extension=='.gif'):
        return tf.image.decode_gif(img_raw)
    else:
        return None

In [ ]:
def load_image(image_path, target_size=[512,512]):
    """Loads, decodes and resizes an image with tensorflow"""
    img_raw = tf.read_file(image_path)
    extension=image_path[-4:].lower()
    img_tensor = decode_according_to_extension(img_raw,extension)
    img_resized = tf.image.resize_images(img_tensor, target_size)
    img_final = img_resized / 255.0
    return img_final

In [ ]:
def one_hot_encode(list_to_encode):
    
    return encoded_array

In [ ]:
# Clean-up
SEED = 42

#ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
tf.set_random_seed(SEED)                             # to keep results consistent (tensorflow seed)

In [ ]:
# Hyperparams
LEARNING_RATE = 0.009
NUM_EPOCHS = 100
MINIBATCH_SIZE = 64

In [ ]:
# load images
path_list = get_images_list('./static/image_data/')
image_list = []
for image in tqdm(path_list):
    image_list.append(load_image(image))

In [ ]:
image_list[1].shape

In [ ]:
# load labels
label_db = sqlite3.connect('halte.db')
cur = label_db.cursor()
cur.execute('SELECT image_id, label FROM image_labels WHERE label_category=\'weapon\' AND user_id=1')
results = cur.fetchall()
images, labels = zip(*results)
label_db.close()
labels = list(labels)
num_classes = len(set(labels))

In [ ]:
# must one-hot encode labels as numpy array
labels_dict = {'none':0,'epee':1,'foil':2,'sabre':3}
labels_int = [labels_dict[l] for l in labels]
labels_array = np.array(labels_int).reshape(-1)
labels_one_hot = np.eye(num_classes)[labels_array]
labels_one_hot

In [ ]:
# define input placeholders

# x for features
# must have a 'None' at the beginning, then image size, then number of channels (check image shape above)
x = tf.placeholder(tf.float32,shape=[None,512,512,3],name='images')
# y for actual classes
# must have a 'None' at the beginning and the other dimension must match number of classes
y = tf.placeholder(tf.float32,shape=[None, num_classes],name='labels') 

# split train / validation / test
# TODO

In [ ]:
# Initialize parameters
# Dimensions are [x_shape of filter, y_shape of filter, # of channels in, # of channels out]
W1 = tf.get_variable('W1', [4,4,3,8], initializer=tf.contrib.layers.xavier_initializer() )
W2 = tf.get_variable('W2', [2,2,8,16], initializer=tf.contrib.layers.xavier_initializer() )

# must give the right dimensions, planning for model layers and sizes

In [ ]:
# Define layers

# CONV2D: stride of 1, padding 'SAME'
Z1 = tf.nn.conv2d(x, W1, strides = [1,1,1,1], padding = 'SAME')
# RELU
A1 = tf.nn.relu(Z1)
# MAXPOOL: window 8x8, stride 8, padding 'SAME'
P1 = tf.nn.max_pool(A1, ksize = [1,8,8,1], strides = [1,8,8,1], padding = 'SAME')
# CONV2D: filters W2, stride 1, padding 'SAME'
Z2 = tf.nn.conv2d(P1, W2, strides = [1,1,1,1], padding = 'SAME')
# RELU
A2 = tf.nn.relu(Z2)
# MAXPOOL: window 4x4, stride 4, padding 'SAME'
P2 = tf.nn.max_pool(A2, ksize = [1,4,4,1], strides = [1,4,4,1], padding = 'SAME')
# FLATTEN
P2 = tf.contrib.layers.flatten(P2)
# FULLY-CONNECTED without non-linear activation function (not not call softmax).
# 6 neurons in output layer. 
Z3 = tf.contrib.layers.fully_connected(P2, num_classes, activation_fn=None)

# y_pred =

In [ ]:
# define cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = y) )

# define optimizer over cost
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(cost)


In [ ]:
# initialize variables
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
# train model
# .... = sess.run((train,loss))

costs = []

# Do the training loop
for epoch in range(NUM_EPOCHS):

    _ , epoch_cost = sess.run([optimizer, cost], feed_dict={x: image_list, y: labels_one_hot})
    costs.append(epoch_cost)

    # Print the cost every epoch
    if epoch % 5 == 0:
        print ("Cost after epoch %i: %f" % (epoch, epoch_cost))


In [ ]:
# run predictions on test dataset (feed_dict x:x_test)


In [ ]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

In [ ]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

In [ ]:
sess = tf.Session()
print(sess.run(total))

In [ ]:
print(sess.run({'ab':(a, b), 'total':total}))

In [ ]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

In [ ]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [ ]:
type(z)

In [ ]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

In [ ]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

In [ ]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

In [ ]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        break

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
sess.run(y,feed_dict={x: [[1, 2, 3],[4, 5, 6]]})

In [ ]:
print(sess.run(y,feed_dict={x: [[1, 0, 0],[0, 0, 0]]}))
print(sess.run(y,feed_dict={x: [[0, 1, 0],[0, 0, 0]]}))
print(sess.run(y,feed_dict={x: [[0, 0, 1],[0, 0, 0]]}))
print(sess.run(y,feed_dict={x: [[0, 0, 0],[1, 0, 0]]}))
print(sess.run(y,feed_dict={x: [[0, 0, 0],[0, 1, 0]]}))
print(sess.run(y,feed_dict={x: [[0, 0, 0],[0, 0, 1]]}))

In [ ]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

In [ ]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

In [ ]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

In [ ]:
print(sess.run(inputs))